# Cloud Botany data access: browse, and download a set

See Jansson et al. [paper](https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2023MS003796). Also notebook templates and [instructions](https://howto.eurec4a.eu/botany_dales.html). From these I downloaded mean profiles, and time-slice 2D fields of TWP LWP and precipitation. Those downloaded subsamples are analyzed in the other notebook in this directory, whose figures appear in the paper. In the profiles, 'wthvt','w2r' are the w - theta-v covariance (buoyancy flux with some unit conversions) and w variance for a quick vertical KE (VKE) budget residence time. 

Environment: `pip install intake xarray intake-xarray zarr pydap s3fs requests ipfsspec` 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pandas.plotting import register_matplotlib_converters
import xarray as xr

In [ ]:
from intake import open_catalog
cat = open_catalog("https://raw.githubusercontent.com/eurec4a/eurec4a-intake/master/catalog.yml")

In [ ]:
# Satellite data from EUREC4A: some other project to look at that ONE MINUTE RES!
# sat = cat.satellites.GOES16.latlongrid.to_dask()
# sat.isel(time=100).plot()     # lat: 2640 lon: 2200 time: 1440 = 1 minute resolution!!

In [ ]:
botany_cat = cat.simulations.DALES.botany

In [ ]:
list(botany_cat.dx100m.nx1536)

-------------------
# 3D fields 

In [8]:
botany_cat.dx100m.nx1536['3D'].to_dask()

,Array,Chunk
Bytes,7.90 TiB,16.00 MiB
Shape,"(103, 60, 149, 1536, 1536)","(1, 8, 8, 256, 256)"
Count,563617 Tasks,563616 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,7.90 TiB,16.00 MiB
Shape,"(103, 60, 149, 1536, 1536)","(1, 8, 8, 256, 256)"
Count,563617 Tasks,563616 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [ ]:
ds_3D = botany_cat.dx100m.nx1536['3D'].to_dask()

# For a spectral KE budget by scale ("configs" = wavenumbers): download some full 3D volumes: 
# Every 6th point in space, every 20th point in time, only up to 3km, only certain cases? 

sub3D = ds_3D[['u','v','w','thl']].isel(time=slice(None, None, 20))\
                                    .isel(xt=slice(None, None, 6))\
                                    .isel(yt=slice(None, None, 6))\
                                    .isel(xm=slice(None, None, 6))\
                                    .isel(ym=slice(None, None, 6))\ 
                                    .sel(zt=slice(0,3000))\    
                                    .sel(zm=slice(0,3000))\ 
                                    .isel(zt=slice(None, None, 6))\    
                                    .isel(zm=slice(None, None, 6))\ 
                                    .sel(member=[   ])


sub3D

# 2D cross section (just a few levels) 

In [10]:
ds_3D = botany_cat.dx100m.nx1536['cross_xy'].to_dask()
ds_3D

,Array,Chunk
Bytes,2.55 TiB,288.00 MiB
Shape,"(4, 103, 720, 1536, 1536)","(1, 1, 32, 1536, 1536)"
Count,9477 Tasks,9476 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,2.55 TiB,288.00 MiB
Shape,"(4, 103, 720, 1536, 1536)","(1, 1, 32, 1536, 1536)"
Count,9477 Tasks,9476 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [16]:
# Every 6th horizontal point (256x256), every 40th time 

sub3D = ds_3D[['uxy','vxy','wxy','thlxy']].isel(time=slice(None, None, 20))\
                                    .isel(xt=slice(None, None, 6))\
                                    .isel(yt=slice(None, None, 6))\
                                    .isel(xm=slice(None, None, 6))\
                                    .isel(ym=slice(None, None, 6)) 
sub3D

,Array,Chunk
Bytes,3.62 GiB,512.00 kiB
Shape,"(4, 103, 36, 256, 256)","(1, 1, 2, 256, 256)"
Count,36669 Tasks,9064 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,3.62 GiB,512.00 kiB
Shape,"(4, 103, 36, 256, 256)","(1, 1, 2, 256, 256)"
Count,36669 Tasks,9064 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [19]:
sub3D.to_netcdf('/Users/bem/Box/EvolutionaryConvection/CloudBotany/uvwthl_4levels_256_36times.nc')

KeyboardInterrupt: 

In [18]:
!ls /Users/bem/Box/EvolutionaryConvection/CloudBotany/

CloudBotany_patterns256.nc             CloudBotany_profiles.nc
CloudBotany_patterns256_UMwifi.nc      CloudBotany_profilesuv.nc
CloudBotany_patterns256_twp_Navwifi.nc Images_cloudwater
CloudBotany_patterns256_twp_UMwifi.nc  Images_cloudwater_hodo


In [ ]:
botany_cat.dx100m.nx1536.